## basics

- RoPE: rotary position embeddings
    - https://arxiv.org/pdf/2104.09864v4.pdf
        - ROFORMER, Jianlin Su
    - 属于 position embedding 的范畴
    - 通过旋转矩阵（rotation matrix）编码绝对位置信息
- 参考
    - https://blog.eleuther.ai/rotary-embeddings/
    - https://sh-tsang.medium.com/brief-review-roformer-enhanced-transformer-with-rotary-position-embedding-36f67a619442

- position encoding：位置编码核心要做的事情

$$
f(\mathbf{x}, \ell)
$$

## freqs_cis